In [1]:
!export SP_DIR=$CONDA_PREFIX/lib/python3.11/site-packages


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random

2023-08-02 17:45:31.666902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 17:45:32.566376: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-02 17:45:33.623671: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# import zipfile
# with zipfile.ZipFile("/home/manogna/Machine-Learning-Collection/ML/Pytorch/more_advanced/Seq2Seq/archive(2).zip", 'r') as zip_ref:
#     zip_ref.extractall("/home/manogna/Machine-Learning-Collection/ML/Pytorch/more_advanced/Seq2Seq/Hindi-English " )


In [4]:
cd /home/manogna/Machine-Learning-Collection/ML/Pytorch/more_advanced/Seq2Seq

/home/manogna/Machine-Learning-Collection/ML/Pytorch/more_advanced/Seq2Seq


In [5]:
# !pip install inltk
from inltk.inltk import tokenize

In [6]:
# from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint

# spacy_ger = spacy.load("de_core_news_sm")
spacy_eng = spacy.load("en_core_web_sm")

In [7]:
def tokenize_hin(text):
    tokens =  tokenize(text,"hi")
    return ' '.join(tokens).lower()


In [8]:
def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]



In [9]:
hindi = Field(tokenize=tokenize_hin, lower=True, init_token="<sos>", eos_token="<eos>")

In [10]:
english = Field(
    tokenize=tokenize_eng, lower=True, init_token="<sos>", eos_token="<eos>"
)


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torchtext
from torchtext.data import Field, TabularDataset


# Read the CSV file using pandas
csv_file = "/home/manogna/Machine-Learning-Collection/ML/Pytorch/more_advanced/Seq2Seq/Hindi-English /hindi_english_parallel.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_file, nrows=50000)

In [12]:
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [13]:
df['english']=df['english'].apply(lambda x: str(x))
df['hindi']=df['hindi'].apply(lambda x: str(x))
df['english']=df['english'].apply(lambda x: x.lower())
df['hindi']=df['hindi'].apply(lambda x: x.lower())

In [14]:
df['hindi'][0]

'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'

In [15]:
import re

In [16]:
df['english']=df['english'].apply(lambda x: re.sub("'", '', x))
df['hindi']=df['hindi'].apply(lambda x: re.sub("'", '', x))

In [17]:
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,the default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,a list of plugins that are disabled by default


In [18]:
import string

In [19]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
df['english']=df['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['hindi']=df['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [20]:
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the top panel
4,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,a list of plugins that are disabled by default


In [21]:
from string import digits

In [22]:
remove_digits = str.maketrans('', '', digits)

In [23]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [24]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
df['english']=df['english'].apply(lambda x: x.translate(remove_digits))
df['hindi']=df['hindi'].apply(lambda x: x.translate(remove_digits))

df['hindi'] = df['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
df['english']=df['english'].apply(lambda x: x.strip())
df['hindi']=df['hindi'].apply(lambda x: x.strip())
df['english']=df['english'].apply(lambda x: re.sub(" +", " ", x))
df['hindi']=df['hindi'].apply(lambda x: re.sub(" +", " ", x))

In [25]:
# Split the data into training, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state=42)

# Convert the dataframes back to CSV files
train_df.to_csv("train_data.csv", index=False, header=False)
valid_df.to_csv("valid_data.csv", index=False, header=False)
test_df.to_csv("test_data.csv", index=False, header=False)

In [26]:
# Create TabularDataset objects for training, validation, and test sets
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
    path="/home/manogna/Machine-Learning-Collection/ML/Pytorch/more_advanced/Seq2Seq", format="csv",
    train="train_data.csv",
    validation="valid_data.csv",
    test="test_data.csv",
    fields=[("hindi", hindi), ("english", english)],
    skip_header=False
)

In [27]:
len(train_data)

36000

In [28]:
hindi.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)



In [29]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        return hidden, cell



In [30]:
class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p
    ):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell



In [31]:
device = "cuda:1"

In [32]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]  #Probable size is (target_len,N)
        target_len = target.shape[0]
        target_vocab_size = len(english.vocab)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs


In [33]:
## We're ready to define everything we need for training our Seq2Seq model ###

# Training hyperparameters
num_epochs = 100
learning_rate = 0.001
batch_size = 64

# Model hyperparameters
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size_encoder = len(hindi.vocab)
input_size_decoder = len(english.vocab)
output_size = len(english.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024  # Needs to be the same for both RNN's
num_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5
print(device)

cuda


In [34]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of available CUDA devices
    num_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_devices)
    
    # Iterate over the available CUDA devices
    for i in range(num_devices):
        device_name = torch.cuda.get_device_name(i)
        print("CUDA Device", i, ":", device_name)
else:
    print("CUDA is not available on this system.")


Number of CUDA devices: 2
CUDA Device 0 : GeForce RTX 2080 Ti
CUDA Device 1 : GeForce RTX 2080 Ti


In [35]:
num_devices = torch.cuda.device_count()
num_devices

2

In [36]:
step = 0


In [37]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.hindi),
    device=device,
)


In [38]:
encoder_net = Encoder(
    input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout
).to(device)

In [39]:
decoder_net = Decoder(
    input_size_decoder,
    decoder_embedding_size,
    hidden_size,
    output_size,
    num_layers,
    dec_dropout,
).to(device)

In [40]:
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [41]:
pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [42]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)


In [43]:
sentence = "खाली बची अतिरिक्त जगह पर कुछ ले जाएँ"

In [44]:
for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    save_checkpoint(checkpoint)

    model.eval()

    translated_sentence = translate_sentence(
        model, sentence, hindi, english, device, max_length=50
    )

    print(f"Translated example sentence: \n {translated_sentence}")

    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.hindi.to(device)
        target = batch.english.to(device)

        # Forward prop
        output = model(inp_data, target)

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin. While we're at it
        # Let's also remove the start token while we're at it
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)

        # Back prop
        loss.backward()

        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # Plot to tensorboard
        # writer.add_scalar("Training loss", loss, global_step=step)
        step += 1


score = bleu(test_data[1:100], model, hindi, english, device)
print(f"Bleu score {score*100:.2f}")


[Epoch 0 / 100]
=> Saving checkpoint
Translated example sentence: 
 ['alidate', 'distance', 'services', 'https', 'enable', 'well', 'well', 'well', 'well', 'prints', 'well', 'well', 'well', 'well', 'kind', 'kind', 'echo', 'echo', 'versions', 'name', 'name', 'mailboxes', 'scope', 'thorough', 'distance', 'services', 'camelot', 'thorough', 'thorough', 'bison', 'mostly', 'several', 'mostly', 'forty', 'five', 'several', 'introduced', 'introduced', 'tommy', 'introduced', 'introduced', 'tommy', 'introduced', 'default', 'default', 'reject', 'gameplay', 'treize', 'extension', 'eyetracker']
[Epoch 1 / 100]
=> Saving checkpoint
Translated example sentence: 
 ['description', '<eos>']
[Epoch 2 / 100]
=> Saving checkpoint
Translated example sentence: 
 ['description', '<eos>']
[Epoch 3 / 100]
=> Saving checkpoint
Translated example sentence: 
 ['page', '<eos>']
[Epoch 4 / 100]
=> Saving checkpoint
Translated example sentence: 
 ['cover', '<eos>']
[Epoch 5 / 100]
=> Saving checkpoint
Translated exampl

KeyError: 'src'

In [47]:
sentence = "खाली बची अतिरिक्त जगह पर कुछ ले जाएँ"

translated_sentence = translate_sentence(
        model, sentence, hindi, english, device, max_length=50)

translated_sentence

['best', '<eos>']